In [1]:
import matplotlib.pyplot as plt
from glob import glob
import face_recognition
import numpy as np
import json
import pickle
import os

In [2]:
inpainted_files = sorted(glob('../../CelebAHQ256_inpainted_gatedconv/*'))

In [3]:
total_num = len(inpainted_files)
total_num

29974

### How often does face_recognition detect a face?
### How often does it correctly identify the face?

In [4]:
# get face encodings for all images
fname_to_encoding = pickle.load(open("../../fname_to_encoding.pkl", 'rb'))
fname_to_id = json.load(open("../../fname_to_id.json", 'r'))
id_to_fname = json.load(open("../../id_to_fname.json", 'r'))

In [5]:
fname_to_encoding

{'../../CelebAHQ256_cleaned/00001.jpg': array([-7.94618055e-02,  1.37288734e-01,  3.25827040e-02, -6.46495149e-02,
        -1.35430649e-01, -3.67258936e-02, -1.48233429e-01, -1.01200394e-01,
         2.10115269e-01, -1.69957832e-01,  1.07220739e-01, -7.19446689e-02,
        -3.24574262e-01,  4.55387868e-02, -1.99087054e-01,  2.62114942e-01,
        -1.79860458e-01, -1.76037759e-01, -6.61247298e-02, -1.32474244e-01,
         3.25718783e-02, -1.55222742e-02,  1.90208126e-02,  1.58829436e-01,
        -1.11335918e-01, -3.60541075e-01,  1.98329473e-03, -5.08325323e-02,
        -8.14255401e-02, -2.18323737e-01, -2.77297292e-03,  7.47526810e-02,
        -1.97019368e-01,  6.03296794e-02, -1.79987103e-02,  2.21446767e-01,
        -6.74327910e-02, -1.64053738e-01,  1.54868752e-01,  9.02637318e-02,
        -2.43026540e-01, -5.31911626e-02,  5.31425551e-02,  2.90603369e-01,
         2.33290121e-01, -2.16227528e-02,  6.95263594e-02, -8.38546753e-02,
         2.19423011e-01, -3.15804392e-01,  1.7459

In [6]:
count_face_detected = 0
count_correctly_identified = 0
count_guessed_correct = 0.
total_to_identify = 0

for f in inpainted_files:
    base_fname = os.path.basename(f)[:9]
    enc_fname = "../../CelebAHQ_cleaned/" + base_fname
    pid = fname_to_id[base_fname][0]
#     print(pid)
#     print(base_fname)
#     print(enc_fname)
    
    img = face_recognition.load_image_file(f)
    face_loc = face_recognition.face_locations(img, model="cnn")
    encoding = face_recognition.face_encodings(img)
    
    # see if a face is detected
    if (len(face_loc) > 0):
        count_face_detected += 1
    
    # see what face_recognition thinks is the most likely face
    # first get degenerate cases out of the way
    if len(encoding) == 0:
        continue
    
    files_with_same_id_that_have_encodings = [f for f in id_to_fname[pid] 
                                              if "../../CelebAHQ256_cleaned/" + f in fname_to_encoding and f != base_fname]
    if len(files_with_same_id_that_have_encodings) == 0:
        continue
    
    total_to_identify += 1
    count_guessed_correct += len(files_with_same_id_that_have_encodings) / (len(fname_to_encoding) - 1)
    
    min_dist = 1e10
    closest_fname = "none"
    for fname in fname_to_encoding:
        if fname == enc_fname:
            continue
        other_encoding = fname_to_encoding[fname]
        dist = face_recognition.face_distance([other_encoding], encoding[0])[0]
        if dist < min_dist:
            min_dist = dist
            closest_fname = fname
    
#     print(min_dist)
#     print(closest_fname)
#     print(fname_to_id[os.path.basename(closest_fname)][0])
    if closest_fname == "none":
        continue
    if fname_to_id[os.path.basename(closest_fname)][0] == pid:
        count_correctly_identified += 1

In [7]:
# how often does it recognize a face
count_face_detected / total_num

0.9995329285380663

In [8]:
# how often does it identify a face by correctly 
# inferring that this face is the same person as 
# another face when there is at least one other face
# with the same identity to compare against
count_correctly_identified / total_to_identify

0.029700135976526158

In [9]:
# random guessing?
count_guessed_correct / total_to_identify

0.0002864648912261707